# Supplementary Tables 1 and 2: Summary Descriptive Tables of all Behavioural Domain and Subdomain Clusters.
## For cerebellum-specfic ALE (C-SALE) and meta-analytic coactivation modeling (MACM).
### Include foci counts, cluter sizes in mm3, and peak coordinates in MNI 2mm space. We also include number of participants and experiments.

#### Author: Neville Magielse
#### Date: 15.08.2024

In [1]:
import os
import sys

# User-specified base working directory
BASE_DIR = '/data/project/cerebellum_ale/'  # Change this to your project folder. Make sure they have input and output folders.

# Defines input and output paths.
INPUT_DIR = os.path.join(BASE_DIR, 'input')
OUTPUT_DIR = os.path.join(BASE_DIR, 'output')

sys.path.append(os.path.join(BASE_DIR, 'scripts'))

In [8]:
# Imports
from pathlib import Path
import nibabel as nib
import nilearn.reporting
import pandas as pd

In [23]:
# Load the CSV data
exp_stats_CSALE = pd.read_csv(os.path.join(OUTPUT_DIR, 'exp_stats_from_json_240814.csv'))

# Optional: If needed, preview the loaded data
print(exp_stats_CSALE.head())

                     subbd  n_exp  n_foci  n_subs
0                   Action    259    1280    3536
1         Action.Execution    198    1033    2544
2  Action.Execution.Speech     47     213     627
3       Action.Imagination     18      55     233
4        Action.Inhibition     36      87     588


In [34]:
# Extract the labels directly from the exp_stats_df
combined_labels = exp_stats_CSALE['subbd'].tolist()
combined_labels

['Action',
 'Action.Execution',
 'Action.Execution.Speech',
 'Action.Imagination',
 'Action.Inhibition',
 'Action.Observation',
 'Cognition',
 'Cognition.Attention',
 'Cognition.Language',
 'Cognition.Language.Orthography',
 'Cognition.Language.Phonology',
 'Cognition.Language.Semantics',
 'Cognition.Language.Speech',
 'Cognition.Memory',
 'Cognition.Memory.Explicit',
 'Cognition.Memory.Working',
 'Cognition.Music',
 'Cognition.Reasoning',
 'Cognition.SocialCognition',
 'Cognition.Spatial',
 'Emotion',
 'Emotion.Negative',
 'Emotion.Negative.Anger',
 'Emotion.Negative.Disgust',
 'Emotion.Negative.Fear',
 'Emotion.Negative.Sadness',
 'Emotion.Positive',
 'Emotion.Positive.Happiness',
 'Emotion.Positive.RewardGain',
 'Interoception',
 'Interoception.Sexuality',
 'Perception',
 'Perception.Audition',
 'Perception.Olfaction',
 'Perception.Somesthesis',
 'Perception.Somesthesis.Pain',
 'Perception.Vision',
 'Perception.Vision.Color',
 'Perception.Vision.Motion',
 'Perception.Vision.Shape']

In [27]:
def load_and_extract_clusters(file_path, label, exp_stats_df):
    # Load the NIfTI file
    img = nib.load(file_path)
    
    # Extract clusters from the z-map
    clusters = nilearn.reporting.get_clusters_table(img, stat_threshold=0.00, cluster_threshold=50)
    
    # Extract the key from the label (removing any additional text)
    key = label.split(": ")[1].strip()  # Extract the domain/subdomain from the label
    
    # Find the corresponding row in exp_stats_df using the subbd column
    match_row = exp_stats_df[exp_stats_df['subbd'] == key]
    
    # Extract number of experiments and participants
    if not match_row.empty:
        n_exp = match_row['n_exp'].values[0]
        n_foci = match_row['n_foci'].values[0]
        n_subs = match_row['n_subs'].values[0]
    else:
        n_exp = 'N/A'
        n_foci = 'N/A'
        n_subs = 'N/A'
    
    # Add the experiments and participants info to the clusters table
    clusters['Experiments'] = n_exp
    clusters['Foci'] = n_foci
    clusters['Participants'] = n_subs
    
    # Store the clusters table
    clusters_table[label] = clusters
    
    # Save clusters to file
    with open(RESULTS_FILE, 'a') as f:
        f.write(f"Results for {label}:\n")
        clusters.to_csv(f, index=False)
        f.write("\n\n")



# Save the C-SALE Cluster Metadata to file.

In [29]:
# Create dictionaries to store variables
meta_result = {}
z_map = {}
clusters_table = {}

# Define domains, subdomains, and granularities (what level are we considering). One can also add activation/ deactiovation or specific tasks as an additionaly granularities.
domains = ['Action', 'Cognition', 'Emotion', 'Interoception', 'Perception']
subdomains = [
    'Execution', 'Execution.Speech', 'Imagination', 'Inhibition', 'MotorLearning', 'Observation', 'Preparation',
    'Attention', 'Language', 'Language.Orthography', 'Language.Phonology', 'Language.Semantics', 'Language.Speech',
    'Language.Syntax', 'Memory', 'Memory.Explicit', 'Memory.Working', 'Music', 'Reasoning', 'SocialCognition',
    'Spatial', 'Temporal', 'Negative', 'Negative.Anger', 'Negative.Anxiety', 'Negative.Disgust', 'Negative.Fear',
    'Negative.Sadness', 'Positive', 'Positive.Happiness', 'Positive.RewardGain', 'Valence', 'Hunger',
    'RespirationRegulation', 'Sexuality', 'Audition', 'Gustation', 'Olfaction', 'Somesthesis', 'Somesthesis.Pain',
    'Vision', 'Vision.Color', 'Vision.Motion', 'Vision.Shape'
]
granularities = ['domain', 'subdomain']

RESULTS_FILE = os.path.join(OUTPUT_DIR, 'C-SALE_clusters-merged.txt') # Decide how you want to name the file.

# Create the results file
with open(RESULTS_FILE, 'w') as f:
    f.write("Cerebellar C-SALE Clusters - Peak Coordinates, Cluster Sizes, and Metadata\n\n")
    
# Loop through domains and granularities to process the Z-maps
for granularity in granularities:
    if (granularity == 'domain'):
        for domain in domains:
            # Define paths for different types of domain-level maps
            nii_path = os.path.join(OUTPUT_DIR, 'SALE', f'{domain}', f'{domain}', 'corr_cluster_h-001_k-50_mask-D2009_MNI_z.nii.gz')
            label = f' Behavioral Domain: {domain}'
            # Load and extract clusters
            try:
                load_and_extract_clusters(nii_path, label, exp_stats_CSALE)
            except FileNotFoundError:
                print(f"File not found for {nii_path}. Skipping...") # Print statement for non-existing BD-level results. Verify that only results without thresholded clusters are omitted.
    else:
        for domain in domains:
            for subdomain in subdomains:
                # Define paths for different types of domain-level maps
                nii_path = os.path.join(OUTPUT_DIR, 'SALE', f'{domain}', f'{domain}.{subdomain}', 'corr_cluster_h-001_k-50_mask-D2009_MNI_z.nii.gz')
                label = f'Behavioral Subdomain: {domain}.{subdomain}'
                # Load and extract clusters
                try:
                    load_and_extract_clusters(nii_path, label, exp_stats_CSALE)
                except FileNotFoundError:
                    print(f"File not found for {nii_path}. Skipping...") # Print statements are added for non-existing subdomain-level results. Note that this may produce a long list of non-existent BD-subdomain combinations.

# Verify the contents of the RESULTS_FILE. They should now include clusters, cluster sizes, peak coordinates, and experiment/ foci/ participamnts numbers for all your domains/ subdomains that have clusters. 


/data/project/cerebellum_ale/venv/lib/python3.11/site-packages/nilearn/reporting/_get_clusters_table.py:103: UserWarning: Attention: At least one of the (sub)peaks falls outside of the cluster body. Identifying the nearest in-cluster voxel.
  warnings.warn(
/data/project/cerebellum_ale/venv/lib/python3.11/site-packages/nilearn/reporting/_get_clusters_table.py:338: UserWarning: Attention: No clusters with stat higher than 0.0
  warnings.warn(
/data/project/cerebellum_ale/venv/lib/python3.11/site-packages/nilearn/reporting/_get_clusters_table.py:338: UserWarning: Attention: No clusters with stat higher than 0.0
  warnings.warn(
/data/project/cerebellum_ale/venv/lib/python3.11/site-packages/nilearn/reporting/_get_clusters_table.py:338: UserWarning: Attention: No clusters with stat higher than 0.0
  warnings.warn(
/data/project/cerebellum_ale/venv/lib/python3.11/site-packages/nilearn/reporting/_get_clusters_table.py:338: UserWarning: Attention: No clusters with stat higher than 0.0
  warni

File not found for /data/project/cerebellum_ale/output/SALE/Action/Action.MotorLearning/corr_cluster_h-001_k-50_mask-D2009_MNI_z.nii.gz. Skipping...


/data/project/cerebellum_ale/venv/lib/python3.11/site-packages/nilearn/reporting/_get_clusters_table.py:103: UserWarning: Attention: At least one of the (sub)peaks falls outside of the cluster body. Identifying the nearest in-cluster voxel.
  warnings.warn(


File not found for /data/project/cerebellum_ale/output/SALE/Action/Action.Preparation/corr_cluster_h-001_k-50_mask-D2009_MNI_z.nii.gz. Skipping...
File not found for /data/project/cerebellum_ale/output/SALE/Action/Action.Attention/corr_cluster_h-001_k-50_mask-D2009_MNI_z.nii.gz. Skipping...
File not found for /data/project/cerebellum_ale/output/SALE/Action/Action.Language/corr_cluster_h-001_k-50_mask-D2009_MNI_z.nii.gz. Skipping...
File not found for /data/project/cerebellum_ale/output/SALE/Action/Action.Language.Orthography/corr_cluster_h-001_k-50_mask-D2009_MNI_z.nii.gz. Skipping...
File not found for /data/project/cerebellum_ale/output/SALE/Action/Action.Language.Phonology/corr_cluster_h-001_k-50_mask-D2009_MNI_z.nii.gz. Skipping...
File not found for /data/project/cerebellum_ale/output/SALE/Action/Action.Language.Semantics/corr_cluster_h-001_k-50_mask-D2009_MNI_z.nii.gz. Skipping...
File not found for /data/project/cerebellum_ale/output/SALE/Action/Action.Language.Speech/corr_clust

/data/project/cerebellum_ale/venv/lib/python3.11/site-packages/nilearn/reporting/_get_clusters_table.py:338: UserWarning: Attention: No clusters with stat higher than 0.0
  warnings.warn(
/data/project/cerebellum_ale/venv/lib/python3.11/site-packages/nilearn/reporting/_get_clusters_table.py:338: UserWarning: Attention: No clusters with stat higher than 0.0
  warnings.warn(
/data/project/cerebellum_ale/venv/lib/python3.11/site-packages/nilearn/reporting/_get_clusters_table.py:338: UserWarning: Attention: No clusters with stat higher than 0.0
  warnings.warn(
/data/project/cerebellum_ale/venv/lib/python3.11/site-packages/nilearn/reporting/_get_clusters_table.py:338: UserWarning: Attention: No clusters with stat higher than 0.0
  warnings.warn(
/data/project/cerebellum_ale/venv/lib/python3.11/site-packages/nilearn/reporting/_get_clusters_table.py:338: UserWarning: Attention: No clusters with stat higher than 0.0
  warnings.warn(


...

/data/project/cerebellum_ale/venv/lib/python3.11/site-packages/nilearn/reporting/_get_clusters_table.py:338: UserWarning: Attention: No clusters with stat higher than 0.0
  warnings.warn(


File not found for /data/project/cerebellum_ale/output/SALE/Interoception/Interoception.Audition/corr_cluster_h-001_k-50_mask-D2009_MNI_z.nii.gz. Skipping...
File not found for /data/project/cerebellum_ale/output/SALE/Interoception/Interoception.Gustation/corr_cluster_h-001_k-50_mask-D2009_MNI_z.nii.gz. Skipping...
File not found for /data/project/cerebellum_ale/output/SALE/Interoception/Interoception.Olfaction/corr_cluster_h-001_k-50_mask-D2009_MNI_z.nii.gz. Skipping...
File not found for /data/project/cerebellum_ale/output/SALE/Interoception/Interoception.Somesthesis/corr_cluster_h-001_k-50_mask-D2009_MNI_z.nii.gz. Skipping...
File not found for /data/project/cerebellum_ale/output/SALE/Interoception/Interoception.Somesthesis.Pain/corr_cluster_h-001_k-50_mask-D2009_MNI_z.nii.gz. Skipping...
File not found for /data/project/cerebellum_ale/output/SALE/Interoception/Interoception.Vision/corr_cluster_h-001_k-50_mask-D2009_MNI_z.nii.gz. Skipping...
File not found for /data/project/cerebell

/data/project/cerebellum_ale/venv/lib/python3.11/site-packages/nilearn/reporting/_get_clusters_table.py:338: UserWarning: Attention: No clusters with stat higher than 0.0
  warnings.warn(


File not found for /data/project/cerebellum_ale/output/SALE/Perception/Perception.Gustation/corr_cluster_h-001_k-50_mask-D2009_MNI_z.nii.gz. Skipping...


/data/project/cerebellum_ale/venv/lib/python3.11/site-packages/nilearn/reporting/_get_clusters_table.py:338: UserWarning: Attention: No clusters with stat higher than 0.0
  warnings.warn(
/data/project/cerebellum_ale/venv/lib/python3.11/site-packages/nilearn/reporting/_get_clusters_table.py:338: UserWarning: Attention: No clusters with stat higher than 0.0
  warnings.warn(
/data/project/cerebellum_ale/venv/lib/python3.11/site-packages/nilearn/reporting/_get_clusters_table.py:338: UserWarning: Attention: No clusters with stat higher than 0.0
  warnings.warn(


# Save the MACM Cluster Metadata to file.

In [31]:
# Load the CSV data
exp_stats_MACM = pd.read_csv(os.path.join(OUTPUT_DIR, 'macm_exp_stats_from_json_240814.csv'))

# Optional: If needed, preview the loaded data
print(exp_stats_MACM.head())

                     subbd  n_exp  n_foci  n_subs
0                   Action    197    3954    2656
1         Action.Execution    251    5195    3451
2  Action.Execution.Speech     80    1638    1096
3       Action.Observation      6      91      92
4       Cognition.Language      9     200     124


In [35]:
# Extract the labels directly from the exp_stats_df
combined_labels_MACM = exp_stats_MACM['subbd'].tolist()
combined_labels_MACM

['Action',
 'Action.Execution',
 'Action.Execution.Speech',
 'Action.Observation',
 'Cognition.Language',
 'Cognition.Memory.Working',
 'Cognition.SocialCognition',
 'Emotion.Negative.Sadness',
 'Perception.Somesthesis',
 'Perception.Vision',
 'Perception.Vision.Motion',
 'Perception.Vision.Shape']

In [39]:
# Define the list of valid subdomains
valid_MACMs = ['Execution', 
               'Execution.Speech', 
               'Memory.Working', 
               'Vision']
# Note that we have to use these labels or get invalid N < 15 Experiments in MACM-analyses results in our table too!

In [40]:
# Create dictionaries
macm_meta_result = {}
macm_z_map = {}
macm_clusters_table = {}

# Define domains and granularities
domains = ['Action', 'Cognition', 'Emotion', 'Perception', 'Interoception']
subdomains = [
    'Execution', 'Execution.Speech', 'Imagination', 'Inhibition', 'MotorLearning', 'Observation', 'Preparation',
    'Attention', 'Language', 'Language.Orthography', 'Language.Phonology', 'Language.Semantics', 'Language.Speech',
    'Language.Syntax', 'Memory', 'Memory.Explicit', 'Memory.Working', 'Music', 'Reasoning', 'SocialCognition',
    'Spatial', 'Temporal', 'Negative', 'Negative.Anger', 'Negative.Anxiety', 'Negative.Disgust', 'Negative.Fear',
    'Negative.Sadness', 'Positive', 'Positive.Happiness', 'Positive.RewardGain', 'Valence', 'Hunger',
    'RespirationRegulation', 'Sexuality', 'Audition', 'Gustation', 'Olfaction', 'Somesthesis', 'Somesthesis.Pain',
    'Vision', 'Vision.Color', 'Vision.Motion', 'Vision.Shape'
]
granularities = ['domain', 'subdomain']

RESULTS_FILE = os.path.join(OUTPUT_DIR,'MACM_clusters-merged.txt') # Decide how you want to name the file.

# Clear the results file
with open(RESULTS_FILE, 'w') as f:
    f.write("Whole Brain MACM Clusters - Peak Coordinates, Cluster Sizes, and Metadata\n\n")
    
# Loop through domains and granularities to process the Z-maps
for granularity in granularities:
    if (granularity == 'domain'):
        for domain in domains:
            # Define paths for different types of domain-level maps
            nii_path = os.path.join(OUTPUT_DIR, 'SALE', f'{domain}', f'{domain}', 'corr_cluster_h-001_k-50_mask-D2009_MNI_z_macm/corr_cluster_k-50_z.nii.gz')
            
            label = f'Behavioral Domain: {domain}'
            # Load and extract clusters
            try:
                load_and_extract_clusters(nii_path, label, exp_stats_MACM)
            except FileNotFoundError:
                print(f"File not found for {nii_path}. Skipping...") # Print statements for non-existing BD-level results. Verify that only results without thresholded clusters are omitted.
    else:
        for domain in domains:
            for subdomain in valid_MACMs:
                # Define paths for different types of domain-level maps
                nii_path = os.path.join(OUTPUT_DIR, 'SALE', f'{domain}', f'{domain}.{subdomain}', 'corr_cluster_h-001_k-50_mask-D2009_MNI_z_macm/corr_cluster_k-50_z.nii.gz')
                label = f'Behavioral Subdomain: {domain}.{subdomain}'
                # Load and extract clusters
                try:
                    load_and_extract_clusters(nii_path, label, exp_stats_MACM)
                except FileNotFoundError:
                    print(f"File not found for {nii_path}. Skipping...") # Print statements for non-existing subdomain-level results. Note that this may produce a long list of non-existent BD-subdomain combinations.

# Verify the contents of the RESULTS_FILE. They should now include clusters, cluster sizes, peak coordinates, and experiment/ foci/ participamnts numbers for all your domains/ subdomains that have clusters. 


/data/project/cerebellum_ale/venv/lib/python3.11/site-packages/nilearn/reporting/_get_clusters_table.py:103: UserWarning: Attention: At least one of the (sub)peaks falls outside of the cluster body. Identifying the nearest in-cluster voxel.
  warnings.warn(


File not found for /data/project/cerebellum_ale/output/SALE/Cognition/Cognition/corr_cluster_h-001_k-50_mask-D2009_MNI_z_macm/corr_cluster_k-50_z.nii.gz. Skipping...
File not found for /data/project/cerebellum_ale/output/SALE/Emotion/Emotion/corr_cluster_h-001_k-50_mask-D2009_MNI_z_macm/corr_cluster_k-50_z.nii.gz. Skipping...
File not found for /data/project/cerebellum_ale/output/SALE/Perception/Perception/corr_cluster_h-001_k-50_mask-D2009_MNI_z_macm/corr_cluster_k-50_z.nii.gz. Skipping...
File not found for /data/project/cerebellum_ale/output/SALE/Interoception/Interoception/corr_cluster_h-001_k-50_mask-D2009_MNI_z_macm/corr_cluster_k-50_z.nii.gz. Skipping...


/data/project/cerebellum_ale/venv/lib/python3.11/site-packages/nilearn/reporting/_get_clusters_table.py:103: UserWarning: Attention: At least one of the (sub)peaks falls outside of the cluster body. Identifying the nearest in-cluster voxel.
  warnings.warn(
/data/project/cerebellum_ale/venv/lib/python3.11/site-packages/nilearn/reporting/_get_clusters_table.py:103: UserWarning: Attention: At least one of the (sub)peaks falls outside of the cluster body. Identifying the nearest in-cluster voxel.
  warnings.warn(


File not found for /data/project/cerebellum_ale/output/SALE/Action/Action.Memory.Working/corr_cluster_h-001_k-50_mask-D2009_MNI_z_macm/corr_cluster_k-50_z.nii.gz. Skipping...
File not found for /data/project/cerebellum_ale/output/SALE/Action/Action.Vision/corr_cluster_h-001_k-50_mask-D2009_MNI_z_macm/corr_cluster_k-50_z.nii.gz. Skipping...
File not found for /data/project/cerebellum_ale/output/SALE/Cognition/Cognition.Execution/corr_cluster_h-001_k-50_mask-D2009_MNI_z_macm/corr_cluster_k-50_z.nii.gz. Skipping...
File not found for /data/project/cerebellum_ale/output/SALE/Cognition/Cognition.Execution.Speech/corr_cluster_h-001_k-50_mask-D2009_MNI_z_macm/corr_cluster_k-50_z.nii.gz. Skipping...


/data/project/cerebellum_ale/venv/lib/python3.11/site-packages/nilearn/reporting/_get_clusters_table.py:103: UserWarning: Attention: At least one of the (sub)peaks falls outside of the cluster body. Identifying the nearest in-cluster voxel.
  warnings.warn(


File not found for /data/project/cerebellum_ale/output/SALE/Cognition/Cognition.Vision/corr_cluster_h-001_k-50_mask-D2009_MNI_z_macm/corr_cluster_k-50_z.nii.gz. Skipping...
File not found for /data/project/cerebellum_ale/output/SALE/Emotion/Emotion.Execution/corr_cluster_h-001_k-50_mask-D2009_MNI_z_macm/corr_cluster_k-50_z.nii.gz. Skipping...
File not found for /data/project/cerebellum_ale/output/SALE/Emotion/Emotion.Execution.Speech/corr_cluster_h-001_k-50_mask-D2009_MNI_z_macm/corr_cluster_k-50_z.nii.gz. Skipping...
File not found for /data/project/cerebellum_ale/output/SALE/Emotion/Emotion.Memory.Working/corr_cluster_h-001_k-50_mask-D2009_MNI_z_macm/corr_cluster_k-50_z.nii.gz. Skipping...
File not found for /data/project/cerebellum_ale/output/SALE/Emotion/Emotion.Vision/corr_cluster_h-001_k-50_mask-D2009_MNI_z_macm/corr_cluster_k-50_z.nii.gz. Skipping...
File not found for /data/project/cerebellum_ale/output/SALE/Perception/Perception.Execution/corr_cluster_h-001_k-50_mask-D2009_MNI

/data/project/cerebellum_ale/venv/lib/python3.11/site-packages/nilearn/reporting/_get_clusters_table.py:103: UserWarning: Attention: At least one of the (sub)peaks falls outside of the cluster body. Identifying the nearest in-cluster voxel.
  warnings.warn(


File not found for /data/project/cerebellum_ale/output/SALE/Interoception/Interoception.Execution/corr_cluster_h-001_k-50_mask-D2009_MNI_z_macm/corr_cluster_k-50_z.nii.gz. Skipping...
File not found for /data/project/cerebellum_ale/output/SALE/Interoception/Interoception.Execution.Speech/corr_cluster_h-001_k-50_mask-D2009_MNI_z_macm/corr_cluster_k-50_z.nii.gz. Skipping...
File not found for /data/project/cerebellum_ale/output/SALE/Interoception/Interoception.Memory.Working/corr_cluster_h-001_k-50_mask-D2009_MNI_z_macm/corr_cluster_k-50_z.nii.gz. Skipping...
File not found for /data/project/cerebellum_ale/output/SALE/Interoception/Interoception.Vision/corr_cluster_h-001_k-50_mask-D2009_MNI_z_macm/corr_cluster_k-50_z.nii.gz. Skipping...


# The End.